In [1]:
import sys
sys.path.append('../bin/')
from load_data import *
import matplotlib.pyplot as plt
from classification import *
import time

In [2]:
input_data = 'mnist'
num_classes= 10
DATA_SHAPE = 28*28

In [3]:
if input_data == 'mnist':
    load_data = load_mnist
if input_data == 'cifar10':
    load_data = load_cifar10

In [4]:
x_train, targets_train, x_test, targets_test = load_data()

In [5]:
def make_plots(x, input_type):

    num = np.int(np.floor(np.sqrt(len(x))))
    if num > 8:
        num = 8
    if input_type=='mnist':
        fig, axes = plt.subplots(num,num,figsize=(num,num))
        plt.set_cmap('gray')
        k =0
        for j in range(num):
            for i in range(num):
                k+=1
                axes[i][j].imshow(x[k].reshape(28,28))
                axes[i][j].axis('off')
        
        plt.show()
        
    elif input_type=='cifar10':
        fig, axes = plt.subplots(num,num,figsize=(num,num))
        k =0
        for j in range(num):
            for i in range(num):
                k+=1
                img = np.swapaxes(np.reshape((x[k]),(32,32,3),'F'),0,1)
                axes[i][j].imshow(img, interpolation='bilinear')
                axes[i][j].axis('off')
        plt.show()
    return True


In [116]:
from sklearn.covariance import LedoitWolf, EmpiricalCovariance, OAS
import scipy.linalg as lg
import numpy.linalg as nlg
from sklearn.decomposition import PCA

def get_covariance(R,var,num):
    '''
    get covarinace estimation for specific number of components
    num: number of components
    R: matrix of eigenvectors
    var: array of eigenvalues
    '''
    fl   = len(var)
    var_ = var[0:num]
    R    = R[0:num]
    
    if num<fl:
        sigma2 = np.mean(var[num::])
    else: 
        sigma2 = 0.

    C_            = np.dot(R.T,np.dot(np.diag(var_), R))+np.eye(len(R.T))*sigma2
    Cinv          = lg.inv(C_)
    sign ,logdetC = nlg.slogdet(C_)
    
    return Cinv, logdetC


class CovarianceEstimator():

    def __init__(self,mode, label):
        
        assert(mode in ['ML','OAS', 'LW','NERCOME'])
        self.mode = mode
        self.label= label
    
    def diag_decomp(self):
        var,R     = lg.eigh(self.cov)
        indices   = np.argsort(var)[::-1]
        self.vars = var[indices]
        self.R    = R.T[indices]
        return self.vars, self.R
    
    def pca(self,data):
        
        num       = data.shape[1]
        pca       = PCA(svd_solver='full',n_components=num)
        pca.fit(data)
        self.pca_vars= pca.explained_variance_
        self.pca_R   = pca.components_
        

    def fit(self,data, mask):

        if self.mode =='ML':
            self.cov = EmpiricalCovariance().fit(data).covariance_
        elif self.mode =='OAS':
            self.cov = OAS().fit(data).covariance_
        elif self.mode =='LW':
            self.cov = LW().fit(data).covariance_
        elif mode == 'NERCOME':
            pass
        else: 
            raise ValueError
        self.mask = mask
        self.mean = np.mean(data, axis=0)

        return True
    
    def compute_logdet(self):
        sign ,self.logdetC = nlg.slogdet(C_)
        return True
    
    def compute_inverse(self):
        self.Cinv = lg.inv(C_)
        return True
    
    def save(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
        self.filename = os.path.join(path,'cov_estimate_%s_%d.pkl'%(self.mode,self.label))
        
        pkl.dump(self, open(self.filename,'wb'))
        
        return self.filename

In [117]:
def prepare_data(data, labels, num_classes, mask_val=0.01):
    '''
    num_classes: number of classes
    '''
    ordered_data = []
    masks   = []
    for ii in range(num_classes):
        ind = np.where(labels==ii)[0]
        d   = data[ind]
        mask= np.where(np.var(d, axis=0)>0.)[0]
        ordered_data+=[d[:,mask]]
        masks+=[mask]
    
    return ordered_data, masks

In [118]:
d_v, m_v = prepare_data(data=x_train,labels=targets_train, num_classes=num_classes)

In [119]:
covs = []
for ii, d in enumerate(d_v):
    print(d.shape)
    cov = CovarianceEstimator(mode='ML',label=ii)
    cov.fit(d,m_v[ii])
    cov.diag_decomp()
    cov.pca(d)
    filename = cov.save('../outputs/%s/covariance_estimator/'%input_data)
    covs+=[cov]

(4932, 563)
(5678, 577)
(4968, 607)
(5101, 581)
(4859, 584)
(4506, 578)
(4951, 558)
(5175, 577)
(4842, 553)
(4988, 545)


In [120]:
def get_log_prob(logdet,Cinv,data,mean):
    d    = len(data)
    data = data-mean
    Cinv_d = np.einsum('jk,...k->...j',Cinv,data, optimize=True)

    logprob = -0.5*logdet-0.5*d*np.log((2*np.pi))-0.5*np.einsum('ij,ij->i',data, Cinv_d, optimize=True)
    return logprob

In [121]:
def classify(data,labels,covs, num_classes, num):
    acc = []
    for jj in range(num_classes):
        indices = np.where(labels==jj)[0]
        dd      = data[indices]
        logprob=[]
        for ii,cov in enumerate(covs):
            if num == dd.shape[1]:
                Cinv    = cov.Cinv
                logdetC = cov.logdetC
            else:
                Cinv, logdetC = get_covariance(cov.R,cov.vars,num)
            d_      = dd[:,cov.mask]
            logprob_= get_log_prob(logdetC,Cinv,d_,cov.mean)
            logprob+=[logprob_]
        acc+=[len(np.where(np.argsort(np.asarray(logprob),axis=0)[-1]==jj)[0])/len(dd)]
    return np.asarray(acc)

In [122]:

accuracy = classify(x_test,targets_test,covs, num_classes, num=200)

In [123]:
accuracy, logprob

(array([ 0.96122449,  0.91894273,  0.97868217,  0.92178218,  0.95723014,
         0.90919283,  0.91858038,  0.92898833,  0.82340862,  0.76808722]),
 array([ 0.96122449,  0.91894273,  0.97868217,  0.92178218,  0.95723014,
         0.90919283,  0.91858038,  0.92898833,  0.82340862,  0.76808722]))

In [124]:
accuracy-logprob

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])